# End to end Weekly Report
I have written this Python Program to automate a 1 Hour 30 Minute manual Excel task, reducing it to 4 Minute for me and my coworker.  

🚀 **Data Analysis & Reporting Project** 📊

### 🎯 **Objective**:
This project focuses on analyzing appointment statuses, patient details, and insurer references, with the goal of generating insightful **Appointment Turnaround Time (TAT)**, **QC TAT**, and **Conversion Rate** reports.

### 🔍 **Key Steps:**

1. **📥 Data Import & Cleaning**:
   - Imported raw data from a CSV file, focusing on key columns related to insurer references and appointment details.
   - Removed unnecessary columns and retained only the most relevant data for analysis.

2. **🔄 Data Transformation**:
   - Filtered and grouped the data based on specific criteria, such as group cases, completed cases, and pending cases.
   - Converted date columns into a proper datetime format to ensure accurate analysis.
   - Created new variables for **Appointment TAT** and **QC TAT**, categorizing them into readable formats (e.g., T+1, T+2).

3. **⏳ Pending & Non-Workable Cases**:
   - Identified pending cases and further categorized them into **Max Attempts**, **DND**, **Non-Contactable**, and more.
   - Separated out non-workable cases (e.g., due to location constraints, order cancellation, etc.) for analysis.

4. **📅 Monthly & Cumulative Reports**:
   - Generated monthly reports showing **cases received**, **non-workable data**, and **conversion rates**.
   - Used **Pivot Tables** to display **Appointment TAT** and **QC TAT** by month and added cumulative sums to track trends.

5. **📈 Conversion Calculations**:
   - Calculated **conversion rates** for **workable data**, showing how many cases were successfully processed compared to the total workable cases.

6. **📑 Final Report Generation**:
   - Consolidated all the results into an **Excel report**, containing multiple sheets for each data point (Raw Data, Cleaned Data, Converted Data, Pending Data, Non-Workable Data, etc.).

### 🛠️ **Technologies Used:**
- **Python**: Data manipulation using **Pandas** and **NumPy**.
- **Libraries**: Pandas, NumPy, datetime, openpyxl (for Excel report generation).

### 📤 **Final Output**:
An automated, structured Excel report summarizing key metrics, which includes:
- **Raw Data**
- **Cleaned Data**
- **Converted Data**
- **Pending Data**
- **Non-Workable Data**
- **Appointment TAT Report**
- **Overall Conversion Report**

This automated process streamlines data analysis and reporting, providing actionable insights at a glance. 

### 💻 **Code Implementation**:

In [19]:
import pandas as pd
import datetime as dt
import numpy as np
from datetime import datetime
from pprint import pprint

# Import Raw Data
Raw_Data=pd.read_csv("mis.csv", low_memory=False)

# It is for the Insurer reference
col = ["CorporateName","RequestDate","PatientName","ApplicationId","OrderID","BookingId","PolicyNo","Age","Gender","RelationShip","EmailId",
       "ContactNo","PackageName","packageInvestigations","ApptCreatedDate","AppointmentDate","ApptTime","SecondPreferredDate",
       "SecondPreferredTime","VisitType","ProviderName","ProviderState","ProviderCity","ProviderLocation",
       "AppointmentStatus","ReportUploaded","reportUrl","QcApprovedDate","QC Approval Month","Photo Available & Type","ClientCity",
       "ClientState","ClientAddress","ClientPincode","AgentName","AgentCode","source","NumberofAttempts","lastCallDateTime","LastCallStatus",
       "ApptCreatedBySelfAllocation","ProductName","planType","loanId","mphName","PackageName","SplitDate","ApprovalType","AM_Name","Escort",
       "PriorityAssigned","DND"]
# It is for the Insurer reference
Raw_Data = Raw_Data[col]

# Relevant Columns to work with
relevant_columns=["RequestDate", "PatientName", "ApplicationId","ReportUploaded", "AppointmentStatus", "LastCallStatus",
                  "AppointmentDate", "QcApprovedDate", "NumberofAttempts", "DND"]

# Remove unnecessary columns and take relevant data in relevant_data
Relevant_Data = Raw_Data[relevant_columns]

# Create group_case_index using .str.startswith("G") and .str.endswith("01")
group_case_index=Relevant_Data[((Relevant_Data["ApplicationId"].str.startswith("G"))&(Relevant_Data["ApplicationId"].str.endswith("01")))
|
((Relevant_Data["ApplicationId"].str.startswith(" G"))&(Relevant_Data["ApplicationId"].str.endswith("01")))].index

# Assign the Group Cases in df Data Frame
df = Relevant_Data.loc[group_case_index]

# Changing the Data Types of Date Columns to Datetime
Date_Columns=["RequestDate", "AppointmentDate", "QcApprovedDate"]
for col in Date_Columns :
    df[col]= pd.to_datetime(df[col], format="%d/%m/%Y") 

# Previous financia years data index
previous_financial_years_data_index=df[df["RequestDate"]<="31/03/2024"].index

#removing Previous financia years data
df=df.drop(previous_financial_years_data_index)

# Index of completed Cases
completed_cases_index=df[df["AppointmentStatus"].isin(["QC Approved", "QC APPROVED", "Reports Uploaded", "Reports Uploaded by DC",
                                                       "Appointment Attended", "QC Rejected", "Sent For Interpretation"])].index

# Converted Data
Converted=df.loc[completed_cases_index, ["RequestDate", "PatientName", "ApplicationId","AppointmentDate","QcApprovedDate","ReportUploaded",
                                         "AppointmentStatus"]]

# Calculate the Appointment TAT and QC TAT and convert it into Object Type to Keep numerical as ell as string values
Converted["Appointment_TAT"]=(Converted["AppointmentDate"]-Converted["RequestDate"]).dt.days.astype("O")
Converted["QC_TAT"]=(Converted["QcApprovedDate"]-Converted["AppointmentDate"]).dt.days.astype("O")

# Replace Null values
Converted["QC_TAT"]=Converted["QC_TAT"].fillna("Pending")

# Define a function Set_TAT_Value to set appropriete TATA value
def Set_TAT_Value(value):
    if isinstance(value, (int, float)) and value<0 :
        return 0
    if isinstance(value, (int, float)) and value==0 :
        return "T0"
    elif isinstance(value, (int, float)) and value==1 :
        return "T+1"
    elif isinstance(value, (int, float)) and value==2 :
        return "T+2"
    elif isinstance(value, (int, float)) and value==3 :
        return "T+3"
    elif isinstance(value, (int, float)) and value==4 :
        return "T+4"
    elif isinstance(value, (int, float)) and value>4 :
        return ">T+4"
    else :
        return value 

# Apply the fuctions to the "Appointment_TAT" and "QC_TAT"
Converted["Appointment_TAT"] = Converted["Appointment_TAT"].apply(Set_TAT_Value)
Converted["QC_TAT"] = Converted["QC_TAT"].apply(Set_TAT_Value)

# Index of Pending Cases
pending_cases_index=df[~(df["AppointmentStatus"].isin(["QC Approved", "QC APPROVED", "Reports Uploaded",
                                                       "Appointment Attended", "QC Rejected", "Sent For Interpretation"]))].index

# Pending Cases
Pending=df.loc[pending_cases_index,["RequestDate", "PatientName", "ApplicationId", "AppointmentStatus", "LastCallStatus", "NumberofAttempts", "DND"]]

# Creating Status Column and and assigning Max Attempt Cases
Pending.loc[Pending["NumberofAttempts"]>30,"Status"]="Max Attempts"

# DND Cases Index
dnd_cases_index=Pending[(Pending["Status"].isna()) & (Pending["DND"]=="Yes")].index

# Assigning DND Cases to Status Column
Pending.loc[dnd_cases_index,"Status"]="DND"

# Appointment Status Cases Index
appointment_status_cases_index=Pending[(Pending["AppointmentStatus"].isin([
    "Cancelled", "Cancelled by insurer", "Cancelled By Provider", "Appointment Confirmed","Order sent to partner"]))&
(Pending["Status"].isna())].index 

# Assigning Appointment Status Cases to Status Column
Pending.loc[appointment_status_cases_index,"Status"] = Pending.loc[appointment_status_cases_index, "AppointmentStatus"]

# LastCallStatus index
LastCallStatus_index=Pending[Pending["Status"].isna()].index 

# Assigning LastCallStatus to Status Column
Pending.loc[LastCallStatus_index,"Status"]=Pending.loc[LastCallStatus_index,"LastCallStatus"]

# Replacing missing values in Status Column with Non Contactable
Pending["Status"]=Pending["Status"].fillna("Non Contactable")

# Working On It Data
Working_On_It_index=Pending[Pending["Status"].isin(["Appointment Request Received", "Direct Medical", "Location Constraint",
                                                    "Medical Done Report Awaited", "Reminder"])].index

Pending.loc[Working_On_It_index,"Status"]="Working On It"

# Workable Data
workable_index=Pending[Pending["Status"].isin(["Appointment Confirmed", "Callback", "Non Contactable", "Order sent to partner", "Working On It"])].index
Pending.loc[workable_index,"Type"]="Workable"

# Non-Workable Data
Pending.loc[Pending[Pending["Type"].isna()].index,"Type"]="Non_Workable"

# Non Workable Data
Non_Workable_Data=Pending.loc[Pending["Type"]=="Non_Workable"]

# Format "RequestDate" as month names for grouping
Converted['Month'] = Converted['RequestDate'].dt.strftime('%b')

# Apply Pivot_Table function to get desired Output
Pivot_Output = pd.pivot_table(
    Converted,
    values='ApplicationId',          # Use a count for 'ApplicationId' or any other identifier
    index='Appointment_TAT',          # Group by 'Appointment_TAT'
    columns='Month',                  # Columns for each month
    aggfunc='count',                  # Count the number of occurrences
    fill_value=0                      # Fill NaNs with 0
)

# Reset index 
Pivot_Output=Pivot_Output.reset_index(drop=False)

# Remove the column name "Month"
Pivot_Output.columns.name = None

# Reorder the index
Pivot_Output=Pivot_Output.loc[[5,1,2,3,4,0]]

# Calculate and add the cumulative Sum Column
Pivot_Output["Apr1"]=Pivot_Output["Apr"].cumsum()
Pivot_Output["May1"]=Pivot_Output["May"].cumsum()
Pivot_Output["Jun1"]=Pivot_Output["Jun"].cumsum()
Pivot_Output["Jul1"]=Pivot_Output["Jul"].cumsum()
Pivot_Output["Aug1"]=Pivot_Output["Aug"].cumsum()
Pivot_Output["Sep1"]=Pivot_Output["Sep"].cumsum()
Pivot_Output["Oct1"]=Pivot_Output["Oct"].cumsum()
Pivot_Output["Nov1"]=Pivot_Output["Nov"].cumsum()

# Remove the normal count month columns
Pivot_Output.drop(columns=["Apr", "Aug", "Jul", "Jun", "May", "Nov", "Oct", "Sep"], inplace=True)

# Rename the cumulative Sum Column
new_col = { "Apr1" : "Apr",
          "May1" : "May",
          "Jun1" : "Jun",
          "Jul1" : "Jul",
          "Aug1" : "Aug",
          "Sep1" : "Sep",
          "Oct1" : "Oct",
          "Nov1" : "Nov"
         }       

# Rename the cumulative Sum Column
Pivot_Output.rename(columns=new_col, inplace=True)

Per_Col=["Apr_Per", "May_Per", "Jun_Per", "Jul_Per", "Aug_Per", "Sep_Per", "Oct_Per", "Nov_Per"]
Norm_Col=["Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"]

for i,j in zip(Per_Col,Norm_Col) :
    Pivot_Output[i]=Pivot_Output[j]/Pivot_Output.loc[0,j]

# Assign the 
Appt_TAT_Report = Pivot_Output.loc[:,["Apr", "Apr_Per", "May", "May_Per", "Jun", "Jun_Per","Jul", "Jul_Per", "Aug", "Aug_Per", "Sep", "Sep_Per",
                                      "Oct", "Oct_Per", "Nov", "Nov_Per"]]

# Format month from the RequestDate and add new column 'Month'
df["Month"] = df["RequestDate"].dt.strftime("%b")

# Count Cases for Each Month
monthly_cases = df.groupby('Month').size().reset_index(name='Cases Received')

#Sort the Months in Calendar Order
month_order = ["Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"]
monthly_cases['Month'] = pd.Categorical(monthly_cases['Month'], categories=month_order, ordered=True)
monthly_cases = monthly_cases.sort_values('Month')

# Calculate and Append the Grand Total

# Calculate the grand total
grand_total = monthly_cases['Cases Received'].sum()

# Create a DataFrame for the grand total row
grand_total_row = pd.DataFrame({'Month': ['Grand Total'], 'Cases Received': [grand_total]})

# Use pd.concat to add the grand total row to the monthly_cases DataFrame
monthly_cases = pd.concat([monthly_cases, grand_total_row], ignore_index=True)

# Format month from the RequestDate and add new column 'Month'
Non_Workable_Data["Month"] = Non_Workable_Data["RequestDate"].dt.strftime("%b")

# Count Cases for Each Month
NW_monthly_cases = Non_Workable_Data.groupby('Month').size().reset_index(name='Cases Received')

#Sort the Months in Calendar Order
month_order = ["Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"]
NW_monthly_cases['Month'] = pd.Categorical(NW_monthly_cases['Month'], categories=month_order, ordered=True)
NW_monthly_cases = NW_monthly_cases.sort_values('Month')

# Calculate and Append the Grand Total

# Calculate the grand total
grand_total = NW_monthly_cases['Cases Received'].sum()

# Create a DataFrame for the grand total row
grand_total_row = pd.DataFrame({'Month': ['Grand Total'], 'Cases Received': [grand_total]})

# Use pd.concat to add the grand total row to the monthly_cases DataFrame
NW_monthly_cases = pd.concat([NW_monthly_cases, grand_total_row], ignore_index=True)

# Rename the column
NW_monthly_cases = NW_monthly_cases.rename(columns = {"Cases Received" : "Non-Workable"})

# Format month from the RequestDate and add new column 'Month'
Converted["Month"] = Converted["RequestDate"].dt.strftime("%b")

# Count Cases for Each Month
c_monthly_cases = Converted.groupby('Month').size().reset_index(name='Converted Data')

#Sort the Months in Calendar Order
month_order = ["Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"]
c_monthly_cases['Month'] = pd.Categorical(c_monthly_cases['Month'], categories=month_order, ordered=True)
c_monthly_cases = c_monthly_cases.sort_values('Month')

# Calculate and Append the Grand Total

# Calculate the grand total
grand_total = c_monthly_cases['Converted Data'].sum()

# Create a DataFrame for the grand total row
grand_total_row = pd.DataFrame({'Month': ['Grand Total'], 'Converted Data': [grand_total]})

# Use pd.concat to add the grand total row to the monthly_cases DataFrame
c_monthly_cases = pd.concat([c_monthly_cases, grand_total_row], ignore_index=True)

Overall_Conversion = monthly_cases

Overall_Conversion["Non-Workable"] = NW_monthly_cases ["Non-Workable"]

Overall_Conversion["Workable"] = Overall_Conversion["Cases Received"] - Overall_Conversion["Non-Workable"]

Overall_Conversion["Converted Data"] = c_monthly_cases["Converted Data"]

Overall_Conversion["Conversion on Workable Data"] = Overall_Conversion["Converted Data"]/Overall_Conversion["Workable"]

# Get the current date and format it as dd-mm-yyyy
from datetime import datetime
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

# Define the destination path
destination= "Report_" + current_date +".xlsx"
destination

# Create the writer variable
writer=pd.ExcelWriter(destination)
Raw_Data.to_excel(writer, sheet_name="Raw Data", index=False)
df.to_excel(writer, sheet_name="Cleaned Data", index=False)
Converted.to_excel(writer, sheet_name="Converted Data", index=False)
Pending.to_excel(writer, sheet_name="Pending Data", index=False)
Non_Workable_Data.to_excel(writer, sheet_name="Non_Workable Data", index=False)
Appt_TAT_Report.to_excel(writer, sheet_name="Appt_TAT_Report", index=False)
Overall_Conversion.to_excel(writer, sheet_name="Overall_Conversion", index=False)
writer.close()

C:\Users\mailt\AppData\Local\Temp\ipykernel_15444\3931492200.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Non_Workable_Data["Month"] = Non_Workable_Data["RequestDate"].dt.strftime("%b")
